In [1]:
# Step 1: Import essential quantum computing modules
import qiskit
import numpy as np
import matplotlib.pyplot as plt

# Import core Qiskit components
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram, plot_bloch_multivector

print("✅ Quantum computing modules loaded successfully!")
print(f"📦 Qiskit version: {qiskit.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print("🎯 Ready to create quantum circuits!")

Matplotlib is building the font cache; this may take a moment.


✅ Quantum computing modules loaded successfully!
📦 Qiskit version: 2.1.2
🔢 NumPy version: 2.3.2
🎯 Ready to create quantum circuits!


In [ ]:
def quantum_gaussian_distribution(n_qubits=4, shots=1000, target_mean=None, target_variance=None):
    """
    Generate a Gaussian-like distribution using quantum computing.
    
    Parameters:
    - n_qubits: Number of qubits (determines resolution, values 0 to 2^n_qubits - 1)
    - shots: Number of measurements
    - target_mean: Desired mean of the distribution (default: middle of range)
    - target_variance: Desired variance (affects the spread)
    
    Returns:
    - Dictionary with measurement results and statistics
    """
    
    max_value = 2 ** n_qubits - 1
    
    # Set default mean to middle of range
    if target_mean is None:
        target_mean = max_value / 2
    
    # Set default variance
    if target_variance is None:
        target_variance = max_value / 6  # Reasonable default spread
    
    # Create quantum circuit
    qc = QuantumCircuit(n_qubits, n_qubits)
    
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qc.h(i)
    
    # Apply controlled rotations to create Gaussian-like distribution
    # Adjust angles based on target variance
    variance_factor = np.sqrt(target_variance / (max_value / 6))
    
    for i in range(n_qubits - 1):
        # Apply controlled RY rotations with variance-adjusted angles
        angle = (np.pi / (2 ** (i + 2))) / variance_factor
        qc.cry(angle, i, i + 1)
    
    # Add more correlations for better Gaussian shape
    for i in range(n_qubits - 2):
        angle = (np.pi / (2 ** (i + 3))) / variance_factor
        qc.cry(angle, i, i + 2)
    
    # Measure all qubits
    qc.measure_all()
    
    # Use StatevectorSampler for simulation
    sampler = StatevectorSampler()
    job = sampler.run([qc], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    # Convert binary results to decimal and shift to target mean
    gaussian_data = {}
    shift = target_mean - (max_value / 2)
    
    for binary_str, count in counts.items():
        decimal_value = int(binary_str, 2)
        shifted_value = decimal_value + shift
        
        # Keep values in reasonable range
        shifted_value = max(0, min(max_value * 2, shifted_value))
        gaussian_data[int(shifted_value)] = count
    
    return gaussian_data

def calculate_statistics(distribution_data):
    """Calculate mean and variance of the distribution."""
    total_samples = sum(distribution_data.values())
    
    # Calculate mean
    mean = sum(value * count for value, count in distribution_data.items()) / total_samples
    
    # Calculate variance
    variance = sum(count * (value - mean) ** 2 for value, count in distribution_data.items()) / total_samples
    
    # Calculate standard deviation
    std_dev = np.sqrt(variance)
    
    return mean, variance, std_dev

# Test the function with different parameters
print("🎲 Generating Quantum Gaussian Distributions with different parameters...\n")

# Test 1: Default parameters
print("📊 Test 1: Default parameters")
gaussian_results1 = quantum_gaussian_distribution(n_qubits=4, shots=1000)
mean1, var1, std1 = calculate_statistics(gaussian_results1)
print(f"Actual Mean: {mean1:.2f}, Variance: {var1:.2f}, Std Dev: {std1:.2f}")

# Test 2: Custom mean and variance
print("\n📊 Test 2: Target mean=10, variance=4")
gaussian_results2 = quantum_gaussian_distribution(n_qubits=4, shots=1000, target_mean=10, target_variance=4)
mean2, var2, std2 = calculate_statistics(gaussian_results2)
print(f"Actual Mean: {mean2:.2f}, Variance: {var2:.2f}, Std Dev: {std2:.2f}")

# Test 3: High variance (wide spread)
print("\n📊 Test 3: Target mean=8, variance=16 (wide spread)")
gaussian_results3 = quantum_gaussian_distribution(n_qubits=4, shots=1000, target_mean=8, target_variance=16)
mean3, var3, std3 = calculate_statistics(gaussian_results3)
print(f"Actual Mean: {mean3:.2f}, Variance: {var3:.2f}, Std Dev: {std3:.2f}")

# Plot all three distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

distributions = [
    (gaussian_results1, "Default (mean≈7.5, var≈2.5)", mean1, var1),
    (gaussian_results2, f"Target: mean=10, var=4", mean2, var2),
    (gaussian_results3, f"Target: mean=8, var=16", mean3, var3)
]

for i, (data, title, actual_mean, actual_var) in enumerate(distributions):
    values = list(data.keys())
    counts = list(data.values())
    
    axes[i].bar(values, counts, alpha=0.7, color=['blue', 'green', 'red'][i], edgecolor='black')
    axes[i].set_title(f'{title}\nActual: μ={actual_mean:.1f}, σ²={actual_var:.1f}', fontsize=10)
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ All distributions generated successfully!")
print("🔬 Note: Quantum circuits provide approximate Gaussian behavior.")
print("? Higher qubit counts and more sophisticated gate sequences can improve accuracy.")

In [ ]:
def quantum_discrete_velocity_cfd(grid_velocities, grid_temperatures, n_qubits=4, shots=500):
    """
    Convert fluid grid (velocity, temperature) to discrete 3-velocity probability distributions.
    
    For each grid point with (mean_velocity, temperature), compute:
    - f(c=-1): Probability of velocity state -1
    - f(c=0):  Probability of velocity state 0  
    - f(c=+1): Probability of velocity state +1
    
    This is useful for lattice Boltzmann methods in CFD.
    
    Parameters:
    - grid_velocities: 2D array of mean velocities at each grid point
    - grid_temperatures: 2D array of temperatures at each grid point
    - n_qubits: Number of qubits for quantum distribution generation
    - shots: Number of quantum measurements
    
    Returns:
    - Dictionary with discrete probability distributions for each grid point
    """
    
    grid_shape = grid_velocities.shape
    discrete_distributions = {}
    
    print(f"🔄 Converting {grid_shape[0]}×{grid_shape[1]} fluid grid to discrete velocity distributions...")
    
    for i in range(grid_shape[0]):
        for j in range(grid_shape[1]):
            mean_vel = grid_velocities[i, j]
            temperature = grid_temperatures[i, j]
            
            # Generate quantum Maxwell-Boltzmann distribution
            mb_dist = quantum_maxwell_boltzmann_discrete(
                mean_velocity=mean_vel,
                temperature=temperature,
                n_qubits=n_qubits,
                shots=shots
            )
            
            # Convert to discrete 3-velocity probabilities
            f_minus1, f_zero, f_plus1 = convert_to_discrete_velocities(mb_dist, mean_vel, temperature)
            
            discrete_distributions[(i, j)] = {
                'f_c_minus1': f_minus1,
                'f_c_zero': f_zero,
                'f_c_plus1': f_plus1,
                'mean_velocity': mean_vel,
                'temperature': temperature,
                'total_prob': f_minus1 + f_zero + f_plus1  # Should be ≈ 1.0
            }
    
    return discrete_distributions

def quantum_maxwell_boltzmann_discrete(mean_velocity=0, temperature=1, n_qubits=4, shots=500):
    """Generate Maxwell-Boltzmann distribution optimized for discrete velocity conversion."""
    
    # Create quantum circuit
    qc = QuantumCircuit(n_qubits, n_qubits)
    
    # Apply Hadamard gates for superposition
    for i in range(n_qubits):
        qc.h(i)
    
    # Temperature-dependent rotations for Maxwell-Boltzmann shape
    temp_factor = 1.0 / (1.0 + temperature)
    
    # Apply controlled rotations with exponential decay
    for i in range(n_qubits - 1):
        angle = np.pi * temp_factor * np.exp(-i / 2.0)
        qc.cry(angle, i, i + 1)
    
    # Add correlations for better exponential shape
    for i in range(n_qubits - 2):
        angle = np.pi * temp_factor * np.exp(-(i + 1) / 3.0)
        qc.cry(angle, i, i + 2)
    
    qc.measure_all()
    
    # Execute quantum circuit
    sampler = StatevectorSampler()
    job = sampler.run([qc], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    # Convert to velocity distribution
    max_value = 2 ** n_qubits - 1
    velocity_distribution = {}
    
    for binary_str, count in counts.items():
        decimal_value = int(binary_str, 2)
        # Map to velocity range centered at mean_velocity
        normalized_pos = (decimal_value / max_value - 0.5) * 2  # Range: [-1, 1]
        velocity_range = 3.0 * np.sqrt(temperature)  # Temperature-dependent range
        velocity = mean_velocity + normalized_pos * velocity_range
        velocity_distribution[velocity] = count
    
    return velocity_distribution

def convert_to_discrete_velocities(velocity_distribution, mean_velocity, temperature):
    """
    Convert continuous velocity distribution to discrete 3-velocity probabilities.
    
    Maps velocities to discrete states: c ∈ {-1, 0, +1}
    """
    
    total_samples = sum(velocity_distribution.values())
    
    # Initialize counters for discrete velocities
    count_minus1 = 0
    count_zero = 0
    count_plus1 = 0
    
    # Define velocity thresholds for discretization
    # Velocities closer to -1, 0, or +1 are assigned to those states
    threshold = 0.5
    
    for velocity, count in velocity_distribution.items():
        # Determine which discrete velocity this maps to
        if velocity < mean_velocity - threshold:
            count_minus1 += count
        elif velocity > mean_velocity + threshold:
            count_plus1 += count
        else:
            count_zero += count
    
    # Convert to probabilities
    f_minus1 = count_minus1 / total_samples
    f_zero = count_zero / total_samples
    f_plus1 = count_plus1 / total_samples
    
    return f_minus1, f_zero, f_plus1

def visualize_discrete_cfd_grid(discrete_distributions, grid_shape):
    """Visualize the discrete velocity probability distributions across the grid."""
    
    # Extract probability arrays
    f_minus1_grid = np.zeros(grid_shape)
    f_zero_grid = np.zeros(grid_shape)
    f_plus1_grid = np.zeros(grid_shape)
    
    for i in range(grid_shape[0]):
        for j in range(grid_shape[1]):
            data = discrete_distributions[(i, j)]
            f_minus1_grid[i, j] = data['f_c_minus1']
            f_zero_grid[i, j] = data['f_c_zero']
            f_plus1_grid[i, j] = data['f_c_plus1']
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Plot f(c=-1)
    im1 = axes[0,0].imshow(f_minus1_grid, cmap='Blues', origin='lower', vmin=0, vmax=1)
    axes[0,0].set_title('f(c = -1): Probability of Velocity State -1')
    axes[0,0].set_xlabel('X Grid')
    axes[0,0].set_ylabel('Y Grid')
    plt.colorbar(im1, ax=axes[0,0])
    
    # Plot f(c=0)
    im2 = axes[0,1].imshow(f_zero_grid, cmap='Greens', origin='lower', vmin=0, vmax=1)
    axes[0,1].set_title('f(c = 0): Probability of Velocity State 0')
    axes[0,1].set_xlabel('X Grid')
    axes[0,1].set_ylabel('Y Grid')
    plt.colorbar(im2, ax=axes[0,1])
    
    # Plot f(c=+1)
    im3 = axes[1,0].imshow(f_plus1_grid, cmap='Reds', origin='lower', vmin=0, vmax=1)
    axes[1,0].set_title('f(c = +1): Probability of Velocity State +1')
    axes[1,0].set_xlabel('X Grid')
    axes[1,0].set_ylabel('Y Grid')
    plt.colorbar(im3, ax=axes[1,0])
    
    # Plot probability conservation check
    total_prob_grid = f_minus1_grid + f_zero_grid + f_plus1_grid
    im4 = axes[1,1].imshow(total_prob_grid, cmap='viridis', origin='lower')
    axes[1,1].set_title('Total Probability (Should be ≈ 1.0)')
    axes[1,1].set_xlabel('X Grid')
    axes[1,1].set_ylabel('Y Grid')
    plt.colorbar(im4, ax=axes[1,1])
    
    plt.tight_layout()
    plt.show()
    
    return f_minus1_grid, f_zero_grid, f_plus1_grid

# Example: Convert CFD grid to discrete velocity distributions
print("🌊 CFD Example: Converting (velocity, temperature) → discrete velocities f(c=-1,0,+1)")

# Create sample fluid grid (2x2 for faster computation)
grid_size = 2
x = np.linspace(0, 1, grid_size)
y = np.linspace(0, 1, grid_size)
X, Y = np.meshgrid(x, y)

# Sample velocity field (flow pattern)
sample_velocities = np.array([[0.5, -0.3], [0.1, 0.8]])

# Sample temperature field (thermal distribution)
sample_temperatures = np.array([[1.0, 2.0], [1.5, 0.8]])

print(f"📊 Input Grid Velocities:\\n{sample_velocities}")
print(f"🌡️  Input Grid Temperatures:\\n{sample_temperatures}")

# Convert to discrete velocity distributions
discrete_cfd = quantum_discrete_velocity_cfd(
    sample_velocities, 
    sample_temperatures, 
    n_qubits=4, 
    shots=300
)

print("\\n🎯 Discrete Velocity Distributions:")
print("="*50)

for i in range(grid_size):
    for j in range(grid_size):
        data = discrete_cfd[(i, j)]
        print(f"Grid({i},{j}): v={data['mean_velocity']:.2f}, T={data['temperature']:.2f}")
        print(f"  f(c=-1) = {data['f_c_minus1']:.3f}")
        print(f"  f(c=0)  = {data['f_c_zero']:.3f}")
        print(f"  f(c=+1) = {data['f_c_plus1']:.3f}")
        print(f"  Total   = {data['total_prob']:.3f}")
        print()

# Visualize the discrete distributions
f_minus1, f_zero, f_plus1 = visualize_discrete_cfd_grid(discrete_cfd, (grid_size, grid_size))

print("✅ Quantum CFD Discrete Velocity Conversion Complete!")
print("🔬 Applications:")
print("   • Lattice Boltzmann Method (LBM) initialization")
print("   • Discrete velocity models for gas kinetics")
print("   • Simplified CFD with quantum-enhanced distributions")
print("   • Real-time fluid simulation with reduced computational cost")